In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [4]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

In [5]:
# spreadsheet_key = "1sCb1YbQ3-1oiL-cnK0yCkwFcQpvpf0efIngAUeC1ixo"
# book = gc.open_by_key(spreadsheet_key)

In [6]:
headers = {
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    "Referer": "https://www.gunviolencearchive.org/",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

In [7]:
%pip install urllib3
import requests
import urllib.request

Note: you may need to restart the kernel to use updated packages.


In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
request = requests.get(
    "https://reproductiverights.org/wp-json/crr-maps/v1/wal?ver=1.0.4", headers=headers
)
data = request.json()

In [10]:
df = pd.DataFrame.from_dict(data["countries"], orient="index")

In [11]:
df.value_counts("category")

category
1352    79
1350    47
1349    44
1348    20
1351    13
         2
dtype: int64

In [12]:
# df[df["category"] == "1352"]

#### categories 
##### 1352 = on request with varying gestational limits
##### 1351 = broad social econ grounds (2nd most permissive)
##### 1350 = to perserve health
##### 1349 = to save life
##### 1348 = broadly prohibited

In [13]:
import numpy as np

In [14]:
conditions = [
    (df["category"] == "1348"),
    (df["category"] == "1349"),
    (df["category"] == "1350"),
    (df["category"] == "1351"),
    (df["category"] == "1352"),
]

values = [
    "Prohibited broadly",
    "Permitted to protect life of pregnant person",
    "Permitted to protect health",
    "Permitted under broad socioeconomic grounds",
    "Permitted with varying gestational limits",
]

df["abortion_rights_text"] = np.select(conditions, values)

df.head()

,name,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text
AFG,Afghanistan,,https://reproductiverights.org/maps/country/af...,1349,[],[],None,None,{},Permitted to protect life of pregnant person
ALB,Albania,,https://reproductiverights.org/maps/country/al...,1352,[1354],[1371],None,"{'title': 'Albania’s Abortion Provisions', 'ur...",{},Permitted with varying gestational limits
DZA,Algeria,,https://reproductiverights.org/maps/country/al...,1350,[],[1366],None,None,{},Permitted to protect health
AND,Andorra,,https://reproductiverights.org/maps/country/an...,1348,[],[],None,None,{},Prohibited broadly
AGO,Angola,,https://reproductiverights.org/maps/country/an...,1350,[],"[1366, 1367, 1368, 1369, 1371]",None,None,{},Permitted to protect health


In [15]:
df["gestation"] = df["gestation"].str[0].astype(str)
df["indications"] = df["indications"].str[0].astype(str)

In [16]:
df[df["name"] == "Canada"]

,name,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text
CAN,Canada,,https://reproductiverights.org/maps/country/ca...,1352,1363.0,nan,None,None,{},Permitted with varying gestational limits


In [17]:
df[df["gestation"] == "1359.0"]

,name,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text
ISL,Iceland,,https://reproductiverights.org/maps/country/ic...,1352,1359.0,nan,None,None,{},Permitted with varying gestational limits


In [18]:
# 1354.0 =12 weeks
# 1356.0 = 14 weeks
# 1363.0 = "Law does not indicate gestational limit; regulatory mechanisms vary" (includes US)
# 1353.0 = 10 weeks
# 1362.0 = 90 days/3 months
# 1360.0 = 24 weeks
# 1364 = law does not limit pre-viability abortion
# 1355 = 120 days
# 1357 = 18 weeks
# 1358 = 20 weeks
# 1359.0 = 22 weeks
# 1361 = 8 weeks
# 1584 = 16 weeks

In [19]:
15 * 7

105

bc I'm trying to do a numerical category for a chloropleth map, adding categorical numbers for the ones with limits that can only be defined by text<br>
no limit specified (like the US) = 1 
no previability limit (like the netherlands/puerto rico) = 2

In [20]:
conditions = [
    (df["gestation"] == "1354.0"),
    (df["gestation"] == "1356.0"),
    (df["gestation"] == "1363.0"),
    (df["gestation"] == "1353.0"),
    (df["gestation"] == "1362.0"),
    (df["gestation"] == "1360.0"),
    (df["gestation"] == "1364.0"),
    (df["gestation"] == "1355.0"),
    (df["gestation"] == "1357.0"),
    (df["gestation"] == "1358.0"),
    (df["gestation"] == "1359.0"),
    (df["gestation"] == "1361.0"),
    (df["gestation"] == "1584.0"),
]

values = [84, 98, 1, 70, 90, 168, 2, 120, 126, 140, 154, 56, 112]

df["gest_limit_days"] = np.select(conditions, values)

df.head()

,name,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text,gest_limit_days
AFG,Afghanistan,,https://reproductiverights.org/maps/country/af...,1349,nan,nan,None,None,{},Permitted to protect life of pregnant person,0
ALB,Albania,,https://reproductiverights.org/maps/country/al...,1352,1354.0,1371.0,None,"{'title': 'Albania’s Abortion Provisions', 'ur...",{},Permitted with varying gestational limits,84
DZA,Algeria,,https://reproductiverights.org/maps/country/al...,1350,nan,1366.0,None,None,{},Permitted to protect health,0
AND,Andorra,,https://reproductiverights.org/maps/country/an...,1348,nan,nan,None,None,{},Prohibited broadly,0
AGO,Angola,,https://reproductiverights.org/maps/country/an...,1350,nan,1366.0,None,None,{},Permitted to protect health,0


In [21]:
df["gest_limit_weeks"] = df["gest_limit_days"] / 7

In [22]:
df = df.reset_index().rename(columns={"index": "iso"})

Est 60 countries with abortion allowed and gestational limits of 15 weeks or less

In [24]:
df.value_counts("gest_limit_days").reset_index().sort_values("gest_limit_days")

,gest_limit_days,0
0,0,130
3,1,6
6,2,2
8,56,1
4,70,4
1,84,44
5,90,4
2,98,7
9,112,1
10,120,1


In [25]:
1 + 4 + 44 + 4 + 7

60

In [26]:
abortion_policies = df[["iso", "name", "abortion_rights_text"]]
chart_id = "MdPmg"

In [27]:
dw.add_data(chart_id=chart_id, data=abortion_policies)

<Response [204]>

### fact checking the lozier report

In [55]:
# continents w iso codes
continents = pd.read_csv(
    "https://gist.githubusercontent.com/stevewithington/20a69c0b6d2ff846ea5d35e5fc47f26c/raw/13716ceb2f22b5643ce5e7039643c86a0e0c6da6/country-and-continent-codes-list-csv.csv"
).rename(
    columns={
        "Country_Name": "name",
        "Three_Letter_Country_Code": "iso",
        "Continent_Name": "continent",
    }
)

In [54]:
# filter for countries with gestational limits 15 weeks or under
continent_ck = (
    df[(df["gest_limit_days"] > 2) & (df["gest_limit_days"] <= 105)]
    .value_counts(["name", "iso"])
    .reset_index()
)

In [57]:
continent_ck.head()

,name,iso,0
0,Albania,ALB,1
1,Argentina,ARG,1
2,Luxembourg,LUX,1
3,Moldova,MDA,1
4,Mongolia,MNG,1


In [58]:
## getting the continents for each ountry that has a gestational limit of 15 weeks or less
continent_ck.merge(continents, on="iso").value_counts("continent")

continent
Europe           37
Asia             14
Africa            6
South America     4
Oceania           2
North America     1
dtype: int64

In [62]:
### looking at the raw data with continents--what am i missing?
abortion_wcontinents = df.merge(continents, on="iso", how="left")

In [65]:
### have to manually account for kosovo and northern ireland which don't match up on iso codes
### still only brings my count of european countries to 39
abortion_wcontinents[abortion_wcontinents["continent"].isna()]

,iso,name_x,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text,gest_limit_days,gest_limit_weeks,continent,Continent_Code,name_y,Two_Letter_Country_Code,Country_Number
100,KOS,Kosovo,,https://reproductiverights.org/maps/country/ko...,1352,1353.0,1371.0,None,None,{},Permitted with varying gestational limits,70,10.0,NaN,NaN,NaN,NaN,NaN
140,NIR,Northern Ireland,,https://reproductiverights.org/maps/country/no...,1352,1354.0,nan,None,None,{},Permitted with varying gestational limits,84,12.0,NaN,NaN,NaN,NaN,NaN


#### line by line lozier report fact check to id where the discrepancy is
##### Lozier - 5 countries w/ 10 weeks, my count is 4
##### Lozier - 27 countries w/ 12 weeks, my count is 28 (with quite a few different countries)
##### Lozier - 2 countries w/ limits between 12 and 14 weeks (match)
##### Lozier - 5 countries w/ 14 weeks - accurate

Lozier - 39 countries that allow abortions w/ gestational limit 15 weeks and under

In [66]:
### "5 European countries limit elective abortion to 10 weeks’ gestation: Croatia, Portugal, Serbia, Slovenia, Turkey"
### that has changed it's now only 4
### Serbia and Slovenia are now 12 weeks (still w/in the 15 week limit though)
#### add in Kosovo
abortion_wcontinents[
    (abortion_wcontinents["gestation"] == "1353.0")
    & (abortion_wcontinents["continent"] == "Europe")
]

,iso,name_x,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text,gest_limit_days,gest_limit_weeks,continent,Continent_Code,name_y,Two_Letter_Country_Code,Country_Number
43,HRV,Croatia,,https://reproductiverights.org/maps/country/cr...,1352,1353.0,1371.0,None,"{'title': 'Croatia’s Abortion Provisions', 'ur...",{},Permitted with varying gestational limits,70,10.0,Europe,EU,"Croatia, Republic of",HR,191.0
152,PRT,Portugal,,https://reproductiverights.org/maps/country/po...,1352,1353.0,1371.0,None,None,{},Permitted with varying gestational limits,70,10.0,Europe,EU,"Portugal, Portuguese Republic",PT,620.0
195,TUR,Türkiye,,https://reproductiverights.org/maps/country/tu...,1352,1353.0,1370.0,,,{},Permitted with varying gestational limits,70,10.0,Europe,EU,"Turkey, Republic of",TR,792.0


In [69]:
### "27 European countries limit elective abortion to 12 weeks’ gestation:
### Albania, Armenia, Azerbaijan, Belarus, Bosnia and Herzegovina, Bulgaria,
### Cyprus, Czech Republic, Denmark, Estonia, France, Georgia, Greece, Hungary,
### Ireland, Kyrgyzstan, Latvia, Lithuania, Montenegro, Moldova, Northern Ireland,
### North Macedonia, Norway, Russia, Slovakia, Switzerland, Ukraine "
## checking which countries center for repro rights has at 12 weeks
### I'm getting 28 countries (when you include northern ireland)
### they have France which is now 16 weeks, so that would make my count 1 less but then I added Slovenia and Serbia
### I also have Kazahkstan but not Kyrgyzstan
### They're missing San Marino?
abortion_wcontinents[
    (abortion_wcontinents["gestation"] == "1354.0")
    & (abortion_wcontinents["continent"] == "Europe")
].reset_index()

In [70]:
# 2 European countries limit elective abortion between 12 and 14 weeks’ gestation : Austria, Italy
## this checks out
abortion_wcontinents[
    (abortion_wcontinents["gestation"] == "1362.0")
    & (abortion_wcontinents["continent"] == "Europe")
].reset_index()

In [71]:
# 5 European countries limit elective abortion to 14 weeks’ gestation: Belgium, Germany, Luxembourg, Romania, Spain
# this checks out
abortion_wcontinents[
    (abortion_wcontinents["gestation"] == "1356.0")
    & (abortion_wcontinents["continent"] == "Europe")
].reset_index()

,index,iso,name_x,body,url,category,gestation,indications,intro_text,link,country_options,abortion_rights_text,gest_limit_days,gest_limit_weeks,continent,Continent_Code,name_y,Two_Letter_Country_Code,Country_Number
0,18,BEL,Belgium,,https://reproductiverights.org/maps/country/be...,1352,1356.0,nan,None,None,{},Permitted with varying gestational limits,98,14.0,Europe,EU,"Belgium, Kingdom of",BE,56.0
1,71,DEU,Germany,,https://reproductiverights.org/maps/country/ge...,1352,1356.0,nan,None,None,{},Permitted with varying gestational limits,98,14.0,Europe,EU,"Germany, Federal Republic of",DE,276.0
2,111,LUX,Luxembourg,,https://reproductiverights.org/maps/country/lu...,1352,1356.0,nan,None,None,{},Permitted with varying gestational limits,98,14.0,Europe,EU,"Luxembourg, Grand Duchy of",LU,442.0
3,156,ROU,Romania,,https://reproductiverights.org/maps/country/ro...,1352,1356.0,nan,None,"{'title': 'Romania’s Abortion Provisions', 'ur...",{},Permitted with varying gestational limits,98,14.0,Europe,EU,Romania,RO,642.0
4,178,ESP,Spain,,https://reproductiverights.org/maps/country/sp...,1352,1356.0,1371.0,None,"{'title': 'Spain’s Abortion Provisions', 'url'...",{},Permitted with varying gestational limits,98,14.0,Europe,EU,"Spain, Kingdom of",ES,724.0


In [81]:
## Lozier - 8 countries that do not allow abortion upon request
## My count is 7
abortion_wcontinents[
    (abortion_wcontinents["continent"] == "Europe")
    & (
        abortion_wcontinents["abortion_rights_text"]
        != "Permitted with varying gestational limits"
    )
]["name_x"]

3            Andorra
63           Finland
73     Great Britain
109    Liechtenstein
117            Malta
124           Monaco
151           Poland
Name: name_x, dtype: object

In [76]:
2 + 2 + 2 + 3 + 3 + 5 + 27

44